In [2]:
import os
import math
import datetime
import numpy as np
import pandas as pd


import time
import os.path

## step 1 obtain OSM network

In [3]:
"Get the OSM Network"
network_type = 'auto'
import osm2gmns as og
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[-6:] == 'VA.osm':
        net = og.getNetFromOSMFile(file,network_type=(network_type), default_lanes=True, default_speed=True)
        break

og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=os.getcwd())

osm2gmns, version 0.4.2


## step 2 Convert tmc Data into node and link

In [4]:
"Convert TMC Data into GMNS Format"
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[:18] == 'TMC_Identification':
        tmc = pd.read_csv(file)
        break


In [5]:

'''build node.csv'''
print('converting tmc data into gmns format...')


node_tmc = pd.DataFrame()
node_tmc['name'] = None
node_tmc['x_coord'] = None
node_tmc['y_coord'] = None
node_tmc['z_coord'] = None
node_tmc['node_type'] = None
node_tmc['ctrl_type'] = None
node_tmc['zone_id'] = None
node_tmc['parent_node_id'] = None
node_tmc['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
    else:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)


node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude'], \
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'start_longitude'].astype(str) + " " + tmc.loc[i+1,'start_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc.index.name = 'node_id'

node_tmc.index += 100000001 #index from 0

node_tmc.to_csv('node_tmc.csv')
print('node_tmc.csv generated!')

converting tmc data into gmns format...
node_tmc.csv generated!


In [6]:
node_tmc

,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id,geometry
node_id,,,,,,,,,
100000001,110N05711,-77.3715,39.0144,None,tmc_start,None,None,None,POINT (-77.3715 39.0144)
100000002,110-05710,-77.3709,39.0141,None,tmc_start,None,None,None,POINT (-77.3709 39.0141)
100000003,110N05710,-77.3684,39.0127,None,tmc_start,None,None,None,POINT (-77.3684 39.0127)
100000004,110-05709,-77.3605,39.0086,None,tmc_start,None,None,None,POINT (-77.3605 39.0086)
100000005,110-05708,-77.3502,39.0028,None,tmc_start,None,None,None,POINT (-77.3502 39.0028)
...,...,...,...,...,...,...,...,...,...
100000060,110+05710,-77.3500,39.0030,None,tmc_start,None,None,None,POINT (-77.35 39.003)
100000061,110P05710,-77.3609,39.0090,None,tmc_start,None,None,None,POINT (-77.3609 39.009)
100000062,110+05711,-77.3673,39.0125,None,tmc_start,None,None,None,POINT (-77.3673 39.0125)


In [7]:
'''build link_tmc.csv'''
link_tmc = pd.DataFrame()
link_tmc['name'] = None
link_tmc['corridor_id'] = None
link_tmc['corridor_link_order'] = None
link_tmc['from_node_id'] = None
link_tmc['to_node_id'] = None
link_tmc['directed'] = None
link_tmc['geometry_id'] = None
link_tmc['geometry'] = None
link_tmc['dir_flag'] = None
link_tmc['parent_link_id'] = None
link_tmc['length'] = None
link_tmc['grade'] = None
link_tmc['facility_type'] = None
link_tmc['capacity'] = None
link_tmc['free_speed'] = None
link_tmc['lanes'] = None

for i in range(0,len(tmc)):
    link_tmc = link_tmc.append({'name': tmc.loc[i,'tmc'],\
                                'corridor_id': tmc.loc[i,'road']+'_'+tmc.loc[i,'direction'],\
                                'corridor_link_order' : tmc.loc[i,'road_order'],\
                                'from_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'start_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                'to_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'end_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                'directed': 1,\
                                'geometry_id': None,\
                                'geometry': "LINESTRING (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) + "," +\
                                    tmc.loc[i,'end_longitude'].astype(str) +" "+ tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                'dir_flag': 1,\
                                'parent_link_id': None,\
                                'length': tmc.loc[i,'miles'],\
                                'grade': None,\
                                'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                'capacity':None,\
                                'free_speed':None,\
                                'lanes': None}, ignore_index=True)
link_tmc.index.name = 'link_id'
link_tmc.index += 100000001


link_tmc.to_csv('link_tmc.csv')
print('link_tmc.csv generated!')


link_tmc.csv generated!


In [8]:
link_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,,,
100000001,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,1,None,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",1,None,0.038620,None,None,None,None,None
100000002,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,1,None,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",1,None,0.164895,None,None,None,None,None
100000003,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,1,None,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",1,None,0.512158,None,None,None,None,None
100000004,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,1,None,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",1,None,0.682589,None,None,None,None,None
100000005,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,1,None,"LINESTRING (-77.3502 39.0028,-77.3455 39.0005)",1,None,0.296731,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000058,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,1,None,"LINESTRING (-77.346 39.001,-77.35 39.003)",1,None,0.254799,None,None,None,None,None
100000059,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,1,None,"LINESTRING (-77.35 39.003,-77.3609 39.009)",1,None,0.718593,None,None,None,None,None
100000060,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,1,None,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",1,None,0.421999,None,None,None,None,None


## step 3 mapping osm network to tmc network

In [9]:
link_road = pd.read_csv('link.csv', low_memory=False)

In [10]:
def LLs2Dist(lon1, lat1, lon2, lat2): #WGS84 transfer coordinate system to distance(meter) #xy #credit to xtHuang0927
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c * 1000
    return distance

In [11]:
for i in range(len(link_road)):
    lon_list = []
    lat_list = [] 
    link_geometry_list = link_road.loc[i,'geometry'][12:-1].split(", ")
    for link_geometry in link_geometry_list:
        lon_list.append(float(link_geometry.split(" ")[0]))
        lat_list.append(float(link_geometry.split(" ")[1]))
    center_lon = np.mean(lon_list)
    center_lat = np.mean(lat_list)


    distance_list = []
    for j in link_tmc.index:
        lon_tmc_list = []
        lat_tmc_list = []
        link_tmc_geometry_list = link_tmc.loc[j,'geometry'][12:-1].split(",")
        for link_tmc_geometry in link_tmc_geometry_list:
            lon_tmc_list.append(float(link_tmc_geometry.split(" ")[0]))
            lat_tmc_list.append(float(link_tmc_geometry.split(" ")[1]))
        center_tmc_lon = np.mean(lon_tmc_list)
        center_tmc_lat = np.mean(lat_tmc_list)
        
        distance_list.append(LLs2Dist(center_lon, center_lat, center_tmc_lon, center_tmc_lat))
    nearest_index = distance_list.index(min(distance_list))

    link_road.loc[i,'tmc_name'] = link_tmc.iloc[nearest_index]['name']
    link_road.loc[i,'tmc_link_id'] = int(link_tmc.iloc[[nearest_index]].index.values[0])
    link_road.loc[i,'matched_from_node_id'] = int(link_tmc.iloc[nearest_index]['from_node_id'])
    link_road.loc[i,'matched_to_node_id'] = int(link_tmc.iloc[nearest_index]['to_node_id'])





In [34]:
link_road

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway,tmc_name,tmc_link_id,matched_from_node_id,matched_to_node_id
0,NaN,0,5964128,4,5,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1878249 38.9744185, -77.187663...",auto,1,110+05700,100000046.0,100000047.0,100000048.0
1,NaN,1,5964128,5,4,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1869767 38.9755412, -77.187013...",auto,1,110+05700,100000046.0,100000047.0,100000048.0
2,NaN,2,5964129,6,7,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.194886 38.9751513, -77.1950799...",auto,1,110+05700,100000046.0,100000047.0,100000048.0
3,NaN,3,5964129,7,6,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196016...",auto,1,110+05700,100000046.0,100000047.0,100000048.0
4,NaN,4,5964129,7,8,1,85.661050,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196222...",auto,1,110+05700,100000046.0,100000047.0,100000048.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30698,Sugarland Road,31152,943412232,3065,3723,1,74.086594,2,35,NaN,tertiary,5,"LINESTRING (-77.3798798 39.0078527, -77.379143...",auto,1,110N05711,100000001.0,100000001.0,100000002.0
30699,Sugarland Road,31153,943412232,3723,3065,1,74.086594,2,35,NaN,tertiary,5,"LINESTRING (-77.37914309999999 39.0075118, -77...",auto,1,110N05711,100000001.0,100000001.0,100000002.0
30700,Sugarland Road,31154,943412232,3723,7903,1,71.834767,2,35,NaN,tertiary,5,"LINESTRING (-77.37914309999999 39.0075118, -77...",auto,1,110N05711,100000001.0,100000001.0,100000002.0
30701,Sugarland Road,31155,943412232,7903,3723,1,71.834767,2,35,NaN,tertiary,5,"LINESTRING (-77.37844440000001 39.0071617, -77...",auto,1,110N05711,100000001.0,100000001.0,100000002.0


## step 4 generating measurement_osm and measurement_tmc

In [29]:
## reading by detid
files= os.listdir(os.getcwd()) 
for file in files: 
    if file[:7] == 'Reading':
        reading = pd.read_csv(file)
        break

# reading = reading[pd.to_datetime(reading['measurement_tstamp'], format='%Y-%m-%d %H:%M:%S')<datetime.datetime.strptime('2015-04-01 02:00:00', '%Y-%m-%d %H:%M:%S')]
reading = reading.loc[0:500]


reading_dict = {}
gp = reading.groupby('_vatmc_code')
for key, form in gp:
    reading_dict[key] = {
        'measurement_tstamp':form['measurement_tstamp'].tolist(),
        'speed':form['speed'].tolist()
        }

In [37]:
'''build measurement_osm.csv''' 


measurement_osm = pd.DataFrame()
measurement_osm['link_id'] = None
measurement_osm['osm_way_id'] = None
measurement_osm['from_node_id'] = None
measurement_osm['to_node_id'] = None
measurement_osm['tmc_link_id'] = None
measurement_osm['matched_from_node_id'] = None
measurement_osm['matched_to_node_id'] = None
measurement_osm['lanes'] = None
measurement_osm['length'] = None
measurement_osm['time_period'] = None
measurement_osm['date'] = None
measurement_osm['geometry'] = None
measurement_osm['volume'] = None
measurement_osm['speed'] = None
measurement_osm_sub = measurement_osm



for i in link_road.index:
    try:
        reading_dict_selected = reading_dict[link_road['tmc_name'][i]]
        for j in range(0,len(reading_dict_selected['measurement_tstamp']),3):
            measurement_osm_sub = measurement_osm_sub.append({'link_id': link_road['link_id'][i],\
                                            'osm_way_id':link_road['osm_way_id'][i],\
                                            'from_node_id': link_road['from_node_id'][i],\
                                            'to_node_id': link_road['to_node_id'][i],\
                                            'tmc_link_id':int(link_road['tmc_link_id'][i]),\
                                            'matched_from_node_id': int(link_road['matched_from_node_id'][i]),\
                                            'matched_to_node_id': int(link_road['matched_to_node_id'][i]),\
                                            'lanes': link_road['lanes'][i], \
                                            'length': link_road['length'][i], \
                                            'time_period': reading_dict_selected['measurement_tstamp'][j][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j][14:16]+'_'+\
                                                reading_dict_selected['measurement_tstamp'][j+3][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j+3][14:16],\
                                            'date':reading_dict_selected['measurement_tstamp'][j][:10],\
                                            'geometry': link_road['geometry'][i],\
                                            'volume': None,\
                                            'speed': round(np.mean(reading_dict_selected['speed'][j:j+3]))}, ignore_index=True)
        measurement_osm = pd.concat([measurement_osm, measurement_osm_sub], ignore_index=True)
        measurement_osm_sub = pd.DataFrame()
    except:
        measurement_osm = measurement_osm.append({'link_id': link_road['link_id'][i],\
                                            'osm_way_id':link_road['osm_way_id'][i],\
                                            'from_node_id': link_road['from_node_id'][i],\
                                            'to_node_id': link_road['to_node_id'][i],\
                                            'tmc_link_id':int(link_road['tmc_link_id'][i]),\
                                            'matched_from_node_id': int(link_road['matched_from_node_id'][i]),\
                                            'matched_to_node_id': int(link_road['matched_to_node_id'][i]),\
                                            'lanes': link_road['lanes'][i], \
                                            'length': link_road['length'][i], \
                                            'time_period': None,\
                                            'date': None,\
                                            'geometry': link_road['geometry'][i],\
                                            'volume': None,\
                                            'speed': None}, ignore_index=True)

    if i % 2000 == 0 : print(i)

measurement_osm.to_csv('measurement_osm.csv',index = False)
print('measurement_osm.csv generated!')

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
measurement_osm.csv generated!


In [40]:
measurement_osm

,link_id,osm_way_id,from_node_id,to_node_id,utd_link_id,matched_from_node_id,matched_to_node_id,lanes,length,time_period,date,geometry,volume,speed,tmc_link_id
0,0,5964128,4,5,NaN,100000047,100000048,1,146.766935,0000_0015,2015-04-01,"LINESTRING (-77.1878249 38.9744185, -77.187663...",None,40,100000046.0
1,0,5964128,4,5,NaN,100000047,100000048,1,146.766935,0015_0030,2015-04-01,"LINESTRING (-77.1878249 38.9744185, -77.187663...",None,40,100000046.0
2,0,5964128,4,5,NaN,100000047,100000048,1,146.766935,None,None,"LINESTRING (-77.1878249 38.9744185, -77.187663...",None,None,100000046.0
3,1,5964128,5,4,NaN,100000047,100000048,1,146.766935,0000_0015,2015-04-01,"LINESTRING (-77.1869767 38.9755412, -77.187013...",None,40,100000046.0
4,1,5964128,5,4,NaN,100000047,100000048,1,146.766935,0015_0030,2015-04-01,"LINESTRING (-77.1869767 38.9755412, -77.187013...",None,40,100000046.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91310,31155,943412232,7903,3723,NaN,100000001,100000002,2,71.834767,0015_0030,2015-04-01,"LINESTRING (-77.37844440000001 39.0071617, -77...",None,44,100000001.0
91311,31155,943412232,7903,3723,NaN,100000001,100000002,2,71.834767,None,None,"LINESTRING (-77.37844440000001 39.0071617, -77...",None,None,100000001.0
91312,31156,943412233,12608,12607,NaN,100000001,100000002,1,27.352271,0000_0015,2015-04-01,"LINESTRING (-77.3791015 39.0131342, -77.379125...",None,44,100000001.0
91313,31156,943412233,12608,12607,NaN,100000001,100000002,1,27.352271,0015_0030,2015-04-01,"LINESTRING (-77.3791015 39.0131342, -77.379125...",None,44,100000001.0


In [41]:
measurement_osm.head(10)

,link_id,osm_way_id,from_node_id,to_node_id,utd_link_id,matched_from_node_id,matched_to_node_id,lanes,length,time_period,date,geometry,volume,speed,tmc_link_id
0,0,5964128,4,5,NaN,100000047,100000048,1,146.766935,0000_0015,2015-04-01,"LINESTRING (-77.1878249 38.9744185, -77.187663...",None,40,100000046.0
1,0,5964128,4,5,NaN,100000047,100000048,1,146.766935,0015_0030,2015-04-01,"LINESTRING (-77.1878249 38.9744185, -77.187663...",None,40,100000046.0
2,0,5964128,4,5,NaN,100000047,100000048,1,146.766935,None,None,"LINESTRING (-77.1878249 38.9744185, -77.187663...",None,None,100000046.0
3,1,5964128,5,4,NaN,100000047,100000048,1,146.766935,0000_0015,2015-04-01,"LINESTRING (-77.1869767 38.9755412, -77.187013...",None,40,100000046.0
4,1,5964128,5,4,NaN,100000047,100000048,1,146.766935,0015_0030,2015-04-01,"LINESTRING (-77.1869767 38.9755412, -77.187013...",None,40,100000046.0
5,1,5964128,5,4,NaN,100000047,100000048,1,146.766935,None,None,"LINESTRING (-77.1869767 38.9755412, -77.187013...",None,None,100000046.0
6,2,5964129,6,7,NaN,100000047,100000048,1,106.872460,0000_0015,2015-04-01,"LINESTRING (-77.194886 38.9751513, -77.1950799...",None,40,100000046.0
7,2,5964129,6,7,NaN,100000047,100000048,1,106.872460,0015_0030,2015-04-01,"LINESTRING (-77.194886 38.9751513, -77.1950799...",None,40,100000046.0
8,2,5964129,6,7,NaN,100000047,100000048,1,106.872460,None,None,"LINESTRING (-77.194886 38.9751513, -77.1950799...",None,None,100000046.0
9,3,5964129,7,6,NaN,100000047,100000048,1,106.872460,0000_0015,2015-04-01,"LINESTRING (-77.1960675 38.9754343, -77.196016...",None,40,100000046.0


In [ ]:
'''build measurement_tmc.csv''' 

measurement_tmc = pd.DataFrame()
measurement_tmc['tmc_link_id'] = None
measurement_tmc['corridor_id'] = None
measurement_tmc['corridor_link_order'] = None
measurement_tmc['from_node_id'] = None
measurement_tmc['to_node_id'] = None
measurement_tmc['time_period'] = None
measurement_tmc['date'] = None
measurement_tmc['geometry'] = None
measurement_tmc['volume'] = None
measurement_tmc['travel_time'] = None
measurement_tmc['speed'] = None
measurement_tmc['reference_speed'] = None
measurement_tmc['density'] = None
measurement_tmc['queue'] = None
measurement_tmc['notes'] = None



for i in link_tmc.index:
    try:
        reading_dict_selected = reading_dict[link_tmc['name'][i]]
        for j in range(0,len(reading_dict_selected['measurement_tstamp']),3):
            measurement_tmc = measurement_tmc.append({'tmc_link_id': i,\
                                            'corridor_id': link_tmc['corridor_id'][i],\
                                            'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                            'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                            'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                            'time_period': reading_dict_selected['measurement_tstamp'][j][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j][14:16]+'_'+\
                                                reading_dict_selected['measurement_tstamp'][j+3][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j+3][14:16],\
                                            'date': reading_dict_selected['measurement_tstamp'][j][:10],\
                                            'geometry': link_tmc['geometry'][i],\
                                            'volume': None,\
                                            'travel_time': None,\
                                            'speed':round(np.mean(reading_dict_selected['speed'][j:j+3])),\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)

    except:
        measurement_tmc = measurement_tmc.append({'tmc_link_id': i,\
                                        'corridor_id': link_tmc['corridor_id'][i],\
                                        'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                        'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                        'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                        'time_period': None,\
                                        'date': None,\
                                        'geometry': link_tmc['geometry'][i],\
                                        'volume': None,\
                                        'travel_time': None,\
                                        'speed': None,\
                                        'reference_speed': None,\
                                        'density': None,\
                                        'queue': None,\
                                        'notes': None }, ignore_index=True)

measurement_tmc.to_csv('measurement_tmc.csv',index = False)
print('measurement_tmc.csv generated!')

In [ ]:
measurement_tmc.append({'tmc_link_id': i,\
                                            'corridor_id': link_tmc['corridor_id'][i],\
                                            'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                            'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                            'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                            'time_period': reading_dict_selected['measurement_tstamp'][j][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j][14:16]+'_'+\
                                                reading_dict_selected['measurement_tstamp'][j+3][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j+3][14:16],\
                                            'date': reading_dict_selected['measurement_tstamp'][j][:10],\
                                            'geometry': link_tmc['geometry'][i],\
                                            'volume': None,\
                                            'travel_time': None,\
                                            'speed':round(np.mean(reading_dict_selected['speed'][j:j+3])),\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }, ignore_index=True)

In [ ]:
measurement_tmc

In [ ]:
measurement_tmc.head(10)

In [27]:
print( __name__)
print(os.path.dirname(os.path.realpath('__file__')))
print(os.path.realpath('__file__'))
print(os.getcwd())

__main__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/__file__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
